In [ ]:
import numpy as np
import os
from shutil import copyfile
from time import sleep
from matplotlib import pyplot as plt

In [ ]:
import subprocess

# Check if COMSOL is in path.
assert 'COMSOL' in subprocess.check_output("echo %PATH%".split(), shell=True).decode(), 'COMSOL is not in path'

In [ ]:
folder = 'Z:/berens/project_bipolarcells/COMSOL2retsim/'

# <font color='red'> ... ensure this folder is also set in you comsol file.</font>

Check:
- Input files, e.g. tmax (parameters) but also currents
- Output file, e.g. export, or data point selection of compartment

In [ ]:
# Get batch size.
def get_batch_size(verbose=True):
    with open(folder + 'interface/' + '_batch_size') as file:
        batch_size = int(file.read())
    if verbose: print('batch_size = ' + str(batch_size))
    return batch_size

In [ ]:
# Get suffix and predix of samples.
def get_sample_prefix_and_suffix(verbose=True):
    with open(folder + 'interface/' + '_samples_prefix') as file:
        prefix = file.read()
        
    with open(folder + 'interface/' + '_samples_suffix') as file:
        suffix = file.read()
        
    if verbose: print('sample_prefix = ' + prefix + '; sample_suffix = ' + suffix)
        
    return prefix, suffix

In [ ]:
# Get all samples that have prefix and sort them.
def get_samples(prefix, suffix, verbose=True):

    # Load all files in samples folder.
    files = os.listdir(folder + 'comsol_input/samples/')
    
    # Get them unsorted, and only consider files with valid prefix.
    sample_files_unsorted = {}
    for file in files:
        assert file.count('.') == 1, 'must have exactly one dot'
        if file[:len(prefix)] == prefix:
            sample_idx = int(file[len(prefix):file.find(suffix)])
            sample_files_unsorted[sample_idx] = file
        else:
            if verbose:
                print(file + ' in samples but different prefix')
            
    # Sort files.
    sample_files = {}
    for sample_idx in sorted(list(sample_files_unsorted.keys())):
        sample_files[sample_idx] = sample_files_unsorted[sample_idx]
        
    return sample_files

In [ ]:
def write_to_file(data, file):
    
    with open(file, 'w+') as f:
        f.write(str(data))

In [ ]:
def set_input_not_ready(verbose=True):
    file = folder + 'interface/' + '_input_ready'
    if os.path.isfile(file): os.remove(file)

def wait_for_ready_input(verbose=True):
    if verbose: print('Waiting for input ... ')
        
    while (True):
        if '_input_ready' in os.listdir(folder + 'interface/'):
            sleep(0.1)
            set_input_not_ready()
            break
        sleep(0.1)
            
    if verbose: print('Ready!')

In [ ]:
def set_comsol_idle(verbose=True):
    write_to_file(1, folder + 'interface/' + '_comsol_idle')
        
def set_comsol_not_idle(verbose=True):
    write_to_file(0, folder + 'interface/' + '_comsol_idle')

In [ ]:
def set_output_ready(verbose=True):
    write_to_file(1, folder + 'interface/' + '_output_ready')
        
def set_output_not_ready(verbose=True):
    file = folder + 'interface/' + '_output_ready'
    if os.path.isfile(file): os.remove(file)

In [ ]:
# Copy a batch of sample to batch folder where comsol will pick them.
def input_samples2batch(sample_idxs, verbose=True):
    
    sample_idxs = np.asarray(sample_idxs)
    assert sample_idxs.size <= batch_size
    
    samples_folder = folder + 'comsol_input/samples/'
    batch_folder   = folder + 'comsol_input/batch/'
    
    # Copy files.
    for batch_idx, sample_idx in enumerate(sample_idxs):
        sample_file = sample_files[sample_idx]
        batch_file  = sample_prefix + str(batch_idx) + sample_suffix
    
        copyfile(samples_folder + sample_file, batch_folder + batch_file)
        if verbose: print('\tbatch/' + batch_file + ' \t--> ' + 'samples/' + sample_file)
        
    # Copy last file several times if batch is otherwise not full.
    # Outputs will be ignored.
    if sample_idxs.size < batch_size:
        for batch_idx in np.arange(sample_idxs.size, batch_size):
            batch_file = sample_prefix + str(batch_idx) + sample_suffix
            copyfile(samples_folder + sample_file, batch_folder + batch_file)

In [ ]:
# Remove all files in input batch.
def clean_output_batch():
    files = os.listdir(folder + 'comsol_output/batch/')
    for file in files:
        os.remove(folder + 'comsol_output/batch/' + file)
    sleep(0.01)

In [ ]:
# Remove all files in input batch.
def clean_output_samples():
    files = os.listdir(folder + 'comsol_output/samples/')
    for file in files:
        os.remove(folder + 'comsol_output/samples/' + file)
    sleep(0.01)

In [ ]:
from os.path import isfile

# Run COMSOL, everything should be ready by then.
def run_comsol(verbose=True):
    with open(folder + 'interface/'+ '_comsol_filename') as file:
        comsol_filename = 'comsol_models/' + file.read()
        
    assert isfile(comsol_filename), 'comsol models does not exist'
        
    cmd = "comsolbatch -inputfile " + comsol_filename
    if verbose: print('Calling ' + cmd)
    try:
        subprocess.call(cmd.split(), shell=True, stderr=subprocess.STDOUT)
    except subprocess.CalledProcessError as e:
        raise RuntimeError("command '{}' return with error (code {}): {}".format(e.cmd, e.returncode, e.output))

In [ ]:
# Remove all files in input batch.
def clean_input_batch():
    files = os.listdir(folder + 'comsol_input/batch/')
    for file in files:
        os.remove(folder + 'comsol_input/batch/' + file)
    sleep(0.01)

In [ ]:
def output_batch2samples(sample_idxs, verbose=True):
    sample_idxs = np.asarray(sample_idxs)
    assert sample_idxs.size <= batch_size

    batch_files = sorted(os.listdir(folder + 'comsol_output/batch/'))
    
    assert len(batch_files) % batch_size == 0
    n_file_types = int(len(batch_files) / batch_size)
    file_type_prefixes = []
    file_type_suffixes = []
    
    # Get prefixes and suffixes of file types.
    for file_type_idx in range(n_file_types):
        
        # Get filename.
        file = batch_files[file_type_idx*batch_size]
        
        # Get prefix.
        file_type_prefix = file[:file.find('_idx')]
        file_type_prefixes.append(file_type_prefix)
        
        # Get suffix.
        file_type_suffix = file[file.find('.'):]
        file_type_suffixes.append(file_type_suffix)
        
    # Copy files from batch to samples.
    batch_folder = folder + 'comsol_output/batch/'
    samples_folder = folder + 'comsol_output/samples/'
    
    for file_type_idx in range(n_file_types):
        
        # Get suffix and prefix.
        file_type_prefix = file_type_prefixes[file_type_idx]
        file_type_suffix = file_type_suffixes[file_type_idx]
        
        # Copy files.
        for batch_idx, sample_idx in enumerate(sample_idxs):
            batch_file  = file_type_prefix + '_idx_' + str(batch_idx) + file_type_suffix
            sample_file = file_type_prefix + '_idx_' + str(sample_idx) + file_type_suffix
            copyfile(batch_folder + batch_file, samples_folder + sample_file)
            if verbose: print('\tbatch/' + batch_file + ' \t--> ' + 'samples/' + sample_file)
                

In [ ]:
while True:
    # Set COMSOL idle.
    set_comsol_idle()
    # Wait for go signal.
    wait_for_ready_input()
    # Received go signal, set output is not ready.
    set_output_not_ready()
    # Delete old output.
    clean_output_samples()
    
    # Get prefix and suffix of samples.
    sample_prefix, sample_suffix = get_sample_prefix_and_suffix(verbose=False)
    # Get all sample files (use prefix and suffix)
    sample_files = get_samples(prefix=sample_prefix, suffix=sample_suffix, verbose=True)
    
    # Get batch size.
    batch_size = get_batch_size(verbose=False)
    
    # Arange samples to batches with batch size.
    sample_idxs_all = list(sample_files.keys())
    sample_idxs_batches =\
        [sample_idxs_all[i*batch_size:(i+1)*batch_size] for i in range(int(np.ceil(len(sample_idxs_all) / batch_size)))]
    
    # Run in batches.
    for sample_idxs in sample_idxs_batches:
        # Get samples into input batch folder.
        input_samples2batch(sample_idxs=sample_idxs)
        # Clean output batch folder.
        clean_output_batch()
        
        # Run Comsol. As long Comsol runs, it is not idle.
        set_comsol_not_idle()
        #input()
        run_comsol()
        set_comsol_idle()
        
        # Copy output batch to samples.
        output_batch2samples(sample_idxs=sample_idxs)
        
        # Clean input batch.
        clean_input_batch()
        # Clean output batch.
        clean_output_batch()
    
    # After all samples are computed, set output ready.
    sleep(0.2)
    set_output_ready()